In [ ]:
#Assignment 2 ArcPy for GIS
#Shafa Hadaina Prawira Sari

#Importing modules

import arcpy 
import sys
import os

#Creating a function for formatting buffer_value.txt

def formatting_b_value_file(b_value_path):
    try: #Testing the script inside, if there is error, it will print out "File not found"
        with open(b_value_path, "r") as bv_file: #Opening the b_value_path which path will be assigned later on
            buffer_values = bv_file.read().strip().split(",") #Reading the file, strip() is used to remove any whitespaces and split(",") is used to remove comma-delimited format
            formatted_values = [] #Creating an empty list
            for value in buffer_values: #Iterating the value in buffer_value file
                try: #Testing the script inside, if the output is error (in this case, it's ValueError), it will print out the assigned string
                    formatted_value = int(round(float(value))) #Assigned formatting value to float and then rounding it up, after that it will be converted to intergers
                    if formatted_value not in formatted_values: #"If statement" for storing the value in formatted_value to the empty list (formatted_values)
                        formatted_values.append(formatted_value) #The statement won't input double value inside the empty list
                except ValueError:
                    print(f"Non-numerical value '{value}' is found in {b_value_path}")
            return sorted(formatted_values) #Returning the list which already sorted from lowest to highest
    except FileNotFoundError:
        print("File not found")
        sys.exit() #Program is terminated at this point for this function
        
b_value_path = r"C:\Users\VivoBook\Documents\Lakehead\ArcPy\buffer_values.txt" #Assigning the b_value_path
in_gdb = r"C:\Users\VivoBook\Documents\Lakehead\ArcPy\data\jhf.gdb" #Assigning the path of the gdb
roads_fc = "roads" #Assigning the feature class in the gdb which containing strings "roads"

def buffered_fclasses(in_feature_class, buffer_values): #Creating function with parameter of certain feature class and buffer_values
    arcpy.env.workspace = in_gdb #Assigning the workspace with the gdb
    for buffer_distance in buffer_values: #Iterating the buffer_distance within buffer values
        out_feature_class = os.path.join(in_gdb, f"roads_{buffer_distance}buff") #os.path.join() is used to create a path to new feature class within the gdb
        arcpy.analysis.Buffer(in_feature_class, out_feature_class, f"{buffer_distance} Meters") #Creating buffered feature classes

def intersect_fri(in_gdb, buffer_values): #Creating function with parameters of various feature class in the gdb and the buffer values
    arcpy.env.workspace = in_gdb #assigning the workspace with the gdb
    fclasses = arcpy.ListFeatureClasses() #Assigning the "fclasses" with the list of feature classes inside the gdb
    for fc in fclasses: #Looping fc in fclasses
        if "fri" in fc and arcpy.Describe(fc).shapeType == "Polygon": #Conditional statement when there is string "fri" inside the name of fc and the shape of the feature class is polygon, the script below will be executed
            for buffer_distance in buffer_values: #Iterating the buffer_distance in the buffer_values
                buffered_fc = os.path.join(in_gdb, f"roads_{buffer_distance}buff") #Assigning "buffered_fc" with the feature class in gdb, in this case, the feature class which contains the buffer_distance parameter
                out_fc = os.path.join(in_gdb, f"fri_{buffer_distance}b_int") #Assigning the output with formatted strings and in the in_gdb
                arcpy.analysis.Intersect([fc, buffered_fc], out_fc) #Creating intersected features (which are the "fri" feature classes in the gdb and the buffered feature classes) and then storing them in out_fc

buffer_values = formatting_b_value_file(b_value_path) #Assigning buffer_values with the formatted buffer_value file
buffered_fclasses(roads_fc, buffer_values) #calling out the function of buffered_fclasses with the assigned parameters
intersect_fri(in_gdb, buffer_values) #calling out the function of intersect_fri with the assigned parameters